In [ ]:
!pip install weaviate-client

## Test 1 - make sure you can connect

> `TODO` - update the host to a url of your instance

In [ ]:
import weaviate
import os
from dotenv import load_dotenv
from weaviate.classes.init import Auth

load_dotenv()
WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")

# Connect to the local instance deployed with Docker Compose
client = weaviate.connect_to_local(
  host="127.0.0.1", # the address to the learner's instance
  port=8080,        # the default REST port
  grpc_port=50051,  # the default GRPC port
  auth_credentials=Auth.api_key(WEAVIATE_KEY)
)

client.is_ready() # Expected True

In [ ]:
# Display the available modules - ideally you should be able to see 'generative-openai' and 'text2vec-openai'
client.get_meta()

In [ ]:
client.close()

## Test 2 - test your OpenAI configuration
> `TODO: 1` - update the `host` to a url of your instance

> `TODO: 2` - update the `base_url` (twice) to your OpenAI url where you host the models

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_URL = os.getenv("OPENAI_URL")
print(f"OpenAI API Key: {OPENAI_API_KEY}")
print(f"OpenAI URL: {OPENAI_URL}")

In [ ]:
import weaviate
from weaviate.classes.init import Auth

# Connect to the local instance
client = weaviate.connect_to_local(
  host="127.0.0.1", # the address to the learner's instance
  port=8080,
  grpc_port=50051,
  auth_credentials=Auth.api_key(WEAVIATE_KEY),
  headers={
    "X-OpenAI-Api-Key": OPENAI_API_KEY
  }
)

print(client.is_ready())

In [ ]:
from weaviate.classes.config import Configure

if client.collections.exists("TestCollection"):
    client.collections.delete("TestCollection")

client.collections.create(
    name="TestCollection",
    
    # Using local ollama embedding model
    vector_config=Configure.Vectors.text2vec_ollama(
        model="nomic-embed-text",
        api_endpoint=OPENAI_URL
    ),
    
    # Using local ollama generative model
    generative_config=Configure.Generative.ollama(
        model="qwen2.5:0.5b",
        api_endpoint=OPENAI_URL
    ),
)

### Load and vectorize data

In [ ]:
import json

# prepare the sample data
with open("./jeopardy_tiny.json") as file:
    data_10 = json.load(file)

print(json.dumps(data_10[0:2], indent=2))

In [ ]:
test_collection = client.collections.get("TestCollection")
test_collection.data.insert_many(data_10)

### Check the data

In [ ]:
# check the number of objects - should be 10
len(test_collection)

In [ ]:
# Show data preview - with vectors
test_collection = client.collections.get("TestCollection")
response = test_collection.query.fetch_objects(
    limit=4,
    include_vector=True
)

for item in response.objects:
  print(item.properties)
  print(item.vector, '\n')

### Query example – it uses the OpenAI key

In [ ]:
response = test_collection.query.near_text(
  query="African animals",
  limit=2
)

for item in response.objects:
  print(item.properties)

### RAG example

In [ ]:
response = test_collection.generate.near_text(
    query="African animals",
    limit=2,
    single_prompt="Please translate {question} to german"
)

print("=== Source ===")
for item in response.objects:
    print(item.properties)
    print(item.generative.text)

In [ ]:
client.close()